# Import Statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None) 

# Read In Data

In [2]:
spotify = pd.read_csv('../data/final_spotify_pull.csv')
print(spotify.shape)
spotify.head()

(1684, 40)


,Unnamed: 0,artist_name,album_name,song_name,popularity,track_id,track_explicit,danceability,energy,key_x,loudness_x,mode_x,speechiness,acousticness,instrumentalness,liveness,valence,tempo_x,type,id_x,uri,track_href,analysis_url,duration_ms,time_signature_x,num_samples,duration,analysis_sample_rate,end_of_fade_in,start_of_fade_out,loudness_y,tempo_y,tempo_confidence,time_signature_y,time_signature_confidence,key_y,key_confidence,mode_y,mode_confidence,id_y
0,0,Marty Robbins,The Essential Marty Robbins 1951-1982,El Paso City,40,4CIaUS9qVxS6RsQBnC37EU,False,0.597,0.4720,0,-11.721,1,0.0342,0.828,0.000002,0.144,0.561,107.590,audio_features,4CIaUS9qVxS6RsQBnC37EU,spotify:track:4CIaUS9qVxS6RsQBnC37EU,https://api.spotify.com/v1/tracks/4CIaUS9qVxS6...,https://api.spotify.com/v1/audio-analysis/4CIa...,251773,4,5551602,251.77333,22050,0.11723,244.99956,-11.721,107.590,0.767,4,0.708,0,0.914,1,0.788,4CIaUS9qVxS6RsQBnC37EU
1,1,Marty Robbins,The Drifter,Faleena (from El Paso),37,2jqx9Oq9ZErm5ywDblnvHi,False,0.644,0.3520,4,-10.562,1,0.0358,0.874,0.000000,0.140,0.643,96.896,audio_features,2jqx9Oq9ZErm5ywDblnvHi,spotify:track:2jqx9Oq9ZErm5ywDblnvHi,https://api.spotify.com/v1/tracks/2jqx9Oq9ZErm...,https://api.spotify.com/v1/audio-analysis/2jqx...,498387,3,10989426,498.38666,22050,0.24721,492.39075,-10.562,96.896,0.378,3,0.607,4,0.890,1,0.714,2jqx9Oq9ZErm5ywDblnvHi
2,2,Johnny Preston,Golden Selection (Remastered),Running Bear - Remastered,32,0x6gSfnYA91AHPLvULn5NK,False,0.762,0.4190,5,-9.312,0,0.0653,0.744,0.000000,0.144,0.765,119.964,audio_features,0x6gSfnYA91AHPLvULn5NK,spotify:track:0x6gSfnYA91AHPLvULn5NK,https://api.spotify.com/v1/tracks/0x6gSfnYA91A...,https://api.spotify.com/v1/audio-analysis/0x6g...,158213,4,3488604,158.21333,22050,0.00000,151.93398,-9.312,119.964,0.639,4,1.000,5,0.766,0,0.602,0x6gSfnYA91AHPLvULn5NK
3,3,Johnny Preston,Greatest Hits,Running Bear,36,1RYznli2VNO7FCbW1Hq4KM,False,0.772,0.2970,5,-14.679,0,0.0530,0.854,0.000008,0.125,0.822,119.987,audio_features,1RYznli2VNO7FCbW1Hq4KM,spotify:track:1RYznli2VNO7FCbW1Hq4KM,https://api.spotify.com/v1/tracks/1RYznli2VNO7...,https://api.spotify.com/v1/audio-analysis/1RYz...,158200,4,3488310,158.20000,22050,1.13624,151.88753,-14.679,119.987,0.700,4,1.000,5,0.686,0,0.585,1RYznli2VNO7FCbW1Hq4KM
4,4,Mark Dinning,The Lovin' Touch,Teen Angel,29,3PymNAkWROfyEVeYq6XtjD,False,0.584,0.0863,0,-15.537,1,0.0403,0.775,0.000000,0.212,0.460,101.493,audio_features,3PymNAkWROfyEVeYq6XtjD,spotify:track:3PymNAkWROfyEVeYq6XtjD,https://api.spotify.com/v1/tracks/3PymNAkWROfy...,https://api.spotify.com/v1/audio-analysis/3Pym...,158200,4,3488310,158.20000,22050,0.30227,151.19093,-15.537,101.493,0.438,4,0.767,0,0.886,1,0.725,3PymNAkWROfyEVeYq6XtjD


In [3]:
hot100 = pd.read_csv('../data/hot100_60-99.csv')
print(hot100.shape)
hot100.head()

(828, 4)


,Unnamed: 0,Song,Artist(s),year
0,0,El Paso,Marty Robbins,1960
1,1,Running Bear,Johnny Preston,1960
2,2,Teen Angel,Mark Dinning,1960
3,3,Theme from A Summer Place,Percy Faith,1960
4,4,Stuck on You,Elvis Presley,1960


# Data Cleaning

One of the first things I want to do when exploring the data I have is to compare the Billboard Hot 100 dataset with the Spotify dataset to see if there were any songs that were missed. In order to make sure I'm able to get the most accurate comparison, I will first want to clean up the string values in the 'song' and 'artist' columns for both datasets. This clean up will include removing special characters, as well as making all characters lowercase. There is a chance this could alter the song or artist name in a way that misspells the name, however I have the track_id to check the song against the Spotify WebAPI incase there are any issues.

### Hot 100 Dataset

In [4]:
# drop 'Unnamed: 0' col
hot100.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [5]:
# rename hot 100 columns to make them easier to work with
hot100.rename({'Song': 'song', 'Artist(s)': 'artist'}, axis=1, inplace=True)

In [6]:
# Create a function to clean strings in 'Song' and 'Artist(s)' col
# this can be used for both hot100 and for df to remove any special characters
# that may differ between these two datasets
def str_col_cleanup(df, columns):
    # Define the special characters to get rid of
    special_characters = "()-_\"\'\/_"
    
    # loop through columns in a list of columns of type object
    for col in columns:
        # loop through special characters replacing with empty string
        for char in special_characters:
            df[col] = df[col].str.replace(char, '', regex=False)
        # lower all values in the series 
        df[col] = df[col].str.lower()
        
    return df

In [7]:
# run the function on the needed object columns in hot100
hot100 = str_col_cleanup(hot100, ['song', 'artist'])

In [8]:
print(hot100.shape)
hot100.head()

(828, 3)


,song,artist,year
0,el paso,marty robbins,1960
1,running bear,johnny preston,1960
2,teen angel,mark dinning,1960
3,theme from a summer place,percy faith,1960
4,stuck on you,elvis presley,1960


## Spotify Dataset

In [9]:
# specify the columns I want to drop, and drop them
col_to_drop = ['Unnamed: 0', 'id_x', 'uri', 'track_href', 'analysis_url', 'id_y', 'type', 'analysis_sample_rate']
spotify.drop(columns=col_to_drop, axis=1, inplace=True)

In [10]:
# rename columns so they are similar with the hot100 
spotify.rename({'artist_name': 'artist', 'album_name': 'album', 'song_name': 'song'}, axis=1, inplace=True)

In [11]:
# run the cleanup function on the needed object columns in spotify
spotify = str_col_cleanup(spotify, ['song', 'artist'])

Next I want to look through spotify for any duplicate entries that may have been pulled. I will focus on looking for duplicates in the 'track_id' column as these ids are specific to the SpotifyAPI and can be used to easily reference the unique values.

In [12]:
# Show any duplicate entries of songs found in the track_id column
spotify[spotify.duplicated(subset='track_id')]

,artist,album,song,popularity,track_id,track_explicit,danceability,energy,key_x,loudness_x,mode_x,speechiness,acousticness,instrumentalness,liveness,valence,tempo_x,duration_ms,time_signature_x,num_samples,duration,end_of_fade_in,start_of_fade_out,loudness_y,tempo_y,tempo_confidence,time_signature_y,time_signature_confidence,key_y,key_confidence,mode_y,mode_confidence
115,frankie valli & the four seasons,Reunited Live,medley: sherry walk like a man big girls don...,32,3gHci6N1SEb58tTDei9RpH,False,0.501,0.718,0,-7.347,1,0.0379,0.0457,0.000287,0.756,0.815,121.419,299027,4,6593538,299.02667,0.00000,283.48663,-7.347,121.419,0.481,4,0.936,0,0.258,1,0.289
116,frankie valli & the four seasons,Reunited Live,medley: sherry walk like a man big girls don...,32,3gHci6N1SEb58tTDei9RpH,False,0.501,0.718,0,-7.347,1,0.0379,0.0457,0.000287,0.756,0.815,121.419,299027,4,6593538,299.02667,0.00000,283.48663,-7.347,121.419,0.481,4,0.936,0,0.258,1,0.289
117,frankie valli & the four seasons,Reunited Live,medley: sherry walk like a man big girls don...,32,3gHci6N1SEb58tTDei9RpH,False,0.501,0.718,0,-7.347,1,0.0379,0.0457,0.000287,0.756,0.815,121.419,299027,4,6593538,299.02667,0.00000,283.48663,-7.347,121.419,0.481,4,0.936,0,0.258,1,0.289
118,frankie valli & the four seasons,Reunited Live,medley: sherry walk like a man big girls don...,32,3gHci6N1SEb58tTDei9RpH,False,0.501,0.718,0,-7.347,1,0.0379,0.0457,0.000287,0.756,0.815,121.419,299027,4,6593538,299.02667,0.00000,283.48663,-7.347,121.419,0.481,4,0.936,0,0.258,1,0.289
119,frankie valli & the four seasons,Reunited Live,medley: sherry walk like a man big girls don...,32,3gHci6N1SEb58tTDei9RpH,False,0.501,0.718,0,-7.347,1,0.0379,0.0457,0.000287,0.756,0.815,121.419,299027,4,6593538,299.02667,0.00000,283.48663,-7.347,121.419,0.481,4,0.936,0,0.258,1,0.289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,bee gees,Tales From The Brothers Gibb,stayin alive from saturday night fever soundt...,52,79hJaqmVdohltPBNN6BULM,False,0.708,0.567,10,-15.539,0,0.0355,0.0700,0.001050,0.152,0.955,103.606,281667,4,6210750,281.66666,0.22095,274.81976,-15.539,103.606,0.964,4,1.000,10,0.481,0,0.573
941,bee gees,Tales From The Brothers Gibb,stayin alive from saturday night fever soundt...,52,79hJaqmVdohltPBNN6BULM,False,0.708,0.567,10,-15.539,0,0.0355,0.0700,0.001050,0.152,0.955,103.606,281667,4,6210750,281.66666,0.22095,274.81976,-15.539,103.606,0.964,4,1.000,10,0.481,0,0.573
942,bee gees,Tales From The Brothers Gibb,stayin alive from saturday night fever soundt...,52,79hJaqmVdohltPBNN6BULM,False,0.708,0.567,10,-15.539,0,0.0355,0.0700,0.001050,0.152,0.955,103.606,281667,4,6210750,281.66666,0.22095,274.81976,-15.539,103.606,0.964,4,1.000,10,0.481,0,0.573
943,bee gees,Tales From The Brothers Gibb,stayin alive from saturday night fever soundt...,52,79hJaqmVdohltPBNN6BULM,False,0.708,0.567,10,-15.539,0,0.0355,0.0700,0.001050,0.152,0.955,103.606,281667,4,6210750,281.66666,0.22095,274.81976,-15.539,103.606,0.964,4,1.000,10,0.481,0,0.573


There seems to be 75 rows of duplicate entries in spotify. I will remove all these duplicates leaving the first value that appears in the dataframe as these duplicates provide no added benefit to my final recommender.

In [13]:
# remove duplicate entries on the track_id col 
spotify.drop_duplicates(subset='track_id', inplace=True)

In [14]:
# recheck for duplicates to make sure they were removed
spotify[spotify.duplicated(subset='track_id')]

,artist,album,song,popularity,track_id,track_explicit,danceability,energy,key_x,loudness_x,mode_x,speechiness,acousticness,instrumentalness,liveness,valence,tempo_x,duration_ms,time_signature_x,num_samples,duration,end_of_fade_in,start_of_fade_out,loudness_y,tempo_y,tempo_confidence,time_signature_y,time_signature_confidence,key_y,key_confidence,mode_y,mode_confidence


In [15]:
# encode boolean values to 0 and 1
spotify['track_explicit'] = spotify['track_explicit'].astype(int)

In [16]:
print(spotify.shape)
spotify.head()

(1609, 32)


,artist,album,song,popularity,track_id,track_explicit,danceability,energy,key_x,loudness_x,mode_x,speechiness,acousticness,instrumentalness,liveness,valence,tempo_x,duration_ms,time_signature_x,num_samples,duration,end_of_fade_in,start_of_fade_out,loudness_y,tempo_y,tempo_confidence,time_signature_y,time_signature_confidence,key_y,key_confidence,mode_y,mode_confidence
0,marty robbins,The Essential Marty Robbins 1951-1982,el paso city,40,4CIaUS9qVxS6RsQBnC37EU,0,0.597,0.4720,0,-11.721,1,0.0342,0.828,0.000002,0.144,0.561,107.590,251773,4,5551602,251.77333,0.11723,244.99956,-11.721,107.590,0.767,4,0.708,0,0.914,1,0.788
1,marty robbins,The Drifter,faleena from el paso,37,2jqx9Oq9ZErm5ywDblnvHi,0,0.644,0.3520,4,-10.562,1,0.0358,0.874,0.000000,0.140,0.643,96.896,498387,3,10989426,498.38666,0.24721,492.39075,-10.562,96.896,0.378,3,0.607,4,0.890,1,0.714
2,johnny preston,Golden Selection (Remastered),running bear remastered,32,0x6gSfnYA91AHPLvULn5NK,0,0.762,0.4190,5,-9.312,0,0.0653,0.744,0.000000,0.144,0.765,119.964,158213,4,3488604,158.21333,0.00000,151.93398,-9.312,119.964,0.639,4,1.000,5,0.766,0,0.602
3,johnny preston,Greatest Hits,running bear,36,1RYznli2VNO7FCbW1Hq4KM,0,0.772,0.2970,5,-14.679,0,0.0530,0.854,0.000008,0.125,0.822,119.987,158200,4,3488310,158.20000,1.13624,151.88753,-14.679,119.987,0.700,4,1.000,5,0.686,0,0.585
4,mark dinning,The Lovin' Touch,teen angel,29,3PymNAkWROfyEVeYq6XtjD,0,0.584,0.0863,0,-15.537,1,0.0403,0.775,0.000000,0.212,0.460,101.493,158200,4,3488310,158.20000,0.30227,151.19093,-15.537,101.493,0.438,4,0.767,0,0.886,1,0.725


# EDA

First I want to make sure that I have the correct year for each of the songs in the main dataframe. To do this, I will find the matching song from the hot100 dataframe and pull the year from there into the main dataframe. I wanted to do this rather then pull the years from Spotify because this project will be initally based on the billboard hot 100 webscrape I did earlier on. This way all versions of the song will have the same year as what billboard reported.

In [17]:
# Set up an empty list to hold year values as I iterate through the
# dataframes looking for song matches
years_list = []

for i, song_name in spotify['song'].items():
    for j, song in hot100['song'].items():
        # Some songs were named a bit differently on 
        # spotify then wiki so by checking if song is in song_name   
        # rather then equal to it I can catch those ones
        if song in song_name:
            years_list.append(hot100['year'][j])
            break

In [18]:
# Make sure the length of the years_list is the same as the main dataframe 
len(years_list) == spotify.shape[0]

True

In [19]:
# Since they are the same shape I can add a new column for the years
spotify['year'] = years_list

This project is also focusing on the specific decade that the song made it to the Billboard Hot 100. As such I want to make sure I get a column for the decade each song is in.

In [20]:
# source : https://stackoverflow.com/questions/17764619/pandas-dataframe-group-year-index-by-decade
# Set up an empty list to hold decade values 
# as I iterate through the year column
decade = []

for year in spotify['year']:
    decade.append((year // 10)*10)
    
spotify['decade'] = decade

spotify.head()

,artist,album,song,popularity,track_id,track_explicit,danceability,energy,key_x,loudness_x,mode_x,speechiness,acousticness,instrumentalness,liveness,valence,tempo_x,duration_ms,time_signature_x,num_samples,duration,end_of_fade_in,start_of_fade_out,loudness_y,tempo_y,tempo_confidence,time_signature_y,time_signature_confidence,key_y,key_confidence,mode_y,mode_confidence,year,decade
0,marty robbins,The Essential Marty Robbins 1951-1982,el paso city,40,4CIaUS9qVxS6RsQBnC37EU,0,0.597,0.4720,0,-11.721,1,0.0342,0.828,0.000002,0.144,0.561,107.590,251773,4,5551602,251.77333,0.11723,244.99956,-11.721,107.590,0.767,4,0.708,0,0.914,1,0.788,1960,1960
1,marty robbins,The Drifter,faleena from el paso,37,2jqx9Oq9ZErm5ywDblnvHi,0,0.644,0.3520,4,-10.562,1,0.0358,0.874,0.000000,0.140,0.643,96.896,498387,3,10989426,498.38666,0.24721,492.39075,-10.562,96.896,0.378,3,0.607,4,0.890,1,0.714,1960,1960
2,johnny preston,Golden Selection (Remastered),running bear remastered,32,0x6gSfnYA91AHPLvULn5NK,0,0.762,0.4190,5,-9.312,0,0.0653,0.744,0.000000,0.144,0.765,119.964,158213,4,3488604,158.21333,0.00000,151.93398,-9.312,119.964,0.639,4,1.000,5,0.766,0,0.602,1960,1960
3,johnny preston,Greatest Hits,running bear,36,1RYznli2VNO7FCbW1Hq4KM,0,0.772,0.2970,5,-14.679,0,0.0530,0.854,0.000008,0.125,0.822,119.987,158200,4,3488310,158.20000,1.13624,151.88753,-14.679,119.987,0.700,4,1.000,5,0.686,0,0.585,1960,1960
4,mark dinning,The Lovin' Touch,teen angel,29,3PymNAkWROfyEVeYq6XtjD,0,0.584,0.0863,0,-15.537,1,0.0403,0.775,0.000000,0.212,0.460,101.493,158200,4,3488310,158.20000,0.30227,151.19093,-15.537,101.493,0.438,4,0.767,0,0.886,1,0.725,1960,1960


## Songs Per Decade/Year

I want to get an idea of my song distribution among decades and years. Not only will this be good for understanding my data better, I can check for any songs that were missing from the Spotify search by checking against the distribution count for the hot100 dataset.  

In [21]:
# get the count of entries per decade using track_id  
# as the unique identifier to count on
decade_count = spotify.groupby('decade')['track_id'].count()
decade_count

decade
1960    444
1970    595
1980    405
1990    165
Name: track_id, dtype: int64

Immediately I see a problem with the decade 1990. The other decades seem to be fairly consistent with each other, although I suspect the lower number from 1980 indicates some missing data as well.

In [22]:
year_count = spotify.groupby('year')['track_id'].count().sort_values()
year_count.head(15)

year
1992     3
1994     6
1997     7
1998     7
1996     9
1982    14
1993    17
1995    18
1983    20
1999    23
1963    24
1968    27
1980    28
1991    30
1981    31
Name: track_id, dtype: int64

Let's start with the worst offenders, the years in 1990 with under 10 song entries. I'll write a function to compare the entries from each of the years in the spotify dataset with the songs from the hot 100 to determine whether there was trouble with the spotify search, or if that particular year just didn't have many songs on the hot 100.

In [23]:
for song in hot100[hot100['year'] == 1992]['song']:
    if spotify[spotify['year'] == 1992]['song'].isin([song]).any():
        print('Found Song!')
    else:
        print(song)

Found Song!
dont let the sun go down on me
im too sexy
to be with you
save the best for last
jump
ill be there
baby got back
Found Song!
Found Song!
how do you talk to an angel
i will always love you


In [24]:
def compare_years(year, df_1=hot100, df_2=spotify):
    hot100_year_songs = df_1[df_1['year'] == year]['song']
    missing_songs = []
    for song in hot100_year_songs:
        if df_2[df_2['year'] == year]['song'].isin([song]).any():
            pass
        else:
            missing_songs.append(song)
    
    return missing_songs

In [45]:
missing_songs = {}
for year in year_count.index:
    
    missing = compare_years(year)
    
    if len(missing) > 0:
        missing_songs[year] = missing
        
missing_songs_count = []
for year in missing_songs:
    missing_songs_count.append([year , len(missing_songs[year])])

In [57]:
missing_songs_count.sort(reverse=True, key=lambda x: x[1])
missing_songs_count

[[1975, 23],
 [1977, 20],
 [1986, 19],
 [1987, 19],
 [1989, 19],
 [1991, 19],
 [1965, 18],
 [1985, 18],
 [1973, 17],
 [1974, 16],
 [1966, 15],
 [1988, 15],
 [1979, 14],
 [1970, 13],
 [1978, 12],
 [1990, 12],
 [1998, 12],
 [1969, 11],
 [1971, 11],
 [1976, 11],
 [1982, 11],
 [1963, 10],
 [1964, 10],
 [1980, 10],
 [1968, 9],
 [1983, 9],
 [1992, 9],
 [1960, 8],
 [1972, 8],
 [1981, 8],
 [1984, 8],
 [1995, 8],
 [1997, 8],
 [1999, 8],
 [1994, 7],
 [1996, 7],
 [1961, 6],
 [1993, 6],
 [1962, 5],
 [1967, 5]]

In [67]:
for year_count in missing_songs_count[:10]:
    print('------------------')
    print(f"{year_count[0]} ---> {year_count[1]} missing")
    print('------------------')
    for song in missing_songs[year]:
        print(song)
    print('------------------')

------------------
1975 ---> 23 missing
------------------
i will survive
what a fool believes
knock on wood
heart of glass
reunited
good times
sad eyes
dont stop til you get enough
rise
pop muzik
heartache tonight
still
no more tears enough is enough
escape the piña colada song
------------------
------------------
1977 ---> 20 missing
------------------
i will survive
what a fool believes
knock on wood
heart of glass
reunited
good times
sad eyes
dont stop til you get enough
rise
pop muzik
heartache tonight
still
no more tears enough is enough
escape the piña colada song
------------------
------------------
1986 ---> 19 missing
------------------
i will survive
what a fool believes
knock on wood
heart of glass
reunited
good times
sad eyes
dont stop til you get enough
rise
pop muzik
heartache tonight
still
no more tears enough is enough
escape the piña colada song
------------------
------------------
1987 ---> 19 missing
------------------
i will survive
what a fool believes
knock on

Even after only grabbing the first 10 years with the longest list of missing songs, I can see that the inital Spotify pull was incomplete. I will now go through the missing songs and adding them to the dataset.

In [27]:
spotify[spotify['year'] == 1992]['song']

1561                       all 4 love
1563    this used to be my playground
1564                  end of the road
Name: song, dtype: object

In [28]:
hot100[hot100['year'] == 1992]['song']

739                        all 4 love
740    dont let the sun go down on me
741                       im too sexy
742                    to be with you
743            save the best for last
744                              jump
745                      ill be there
746                     baby got back
747     this used to be my playground
748                   end of the road
749       how do you talk to an angel
750            i will always love you
Name: song, dtype: object